In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 300)

In [3]:
game_df = pd.read_pickle('data/games.pickle')

In [4]:
game_df.shape


(20508, 38)

In [5]:
mask = game_df['season_year'] >= 1990
game_df  = game_df[mask]

In [6]:
game_df.columns

Index(['season_year', 'week_num', 'full_game_date', 'team', 'opp',
       'game_day_of_week', 'game_date', 'game_outcome', 'pts_off', 'pts_def',
       'pass_cmp', 'pass_att', 'pass_yds', 'pass_td', 'pass_int',
       'pass_sacked', 'pass_sacked_yds', 'pass_yds_per_att',
       'pass_net_yds_per_att', 'pass_cmp_perc', 'pass_rating', 'rush_att',
       'rush_yds', 'rush_yds_per_att', 'rush_td', 'fgm', 'fga', 'xpm', 'xpa',
       'punt', 'punt_yds', 'overtime', 'game_location', 'time_of_poss',
       'third_down_success', 'third_down_att', 'fourth_down_success',
       'fourth_down_att'],
      dtype='object')

In [7]:
mask = (game_df.team == 'chi') & (game_df.season_year == 2019)
game_df[mask]

,season_year,week_num,full_game_date,team,opp,game_day_of_week,game_date,game_outcome,pts_off,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,game_location,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att
19520,2019,1.0,2019-09-05,chi,gnb,Thu,September 5,L,3.0,10.0,26.0,45.0,208.0,0.0,1.0,5.0,20.0,5.0,4.0,57.0,60.0,15.0,46.0,3.0,0.0,1.0,1.0,0.0,0.0,8.0,341.0,NaN,NaN,28:57,3.0,15.0,0.0,2.0
19521,2019,2.0,2019-09-15,chi,den,Sun,September 15,W,16.0,14.0,16.0,27.0,120.0,0.0,0.0,0.0,0.0,4.0,4.0,59.0,70.0,29.0,153.0,5.0,1.0,3.0,3.0,1.0,1.0,5.0,287.0,NaN,@,27:01,3.0,11.0,1.0,1.0
19522,2019,3.0,2019-09-23,chi,was,Mon,September 23,W,31.0,15.0,25.0,31.0,208.0,3.0,1.0,3.0,23.0,7.0,6.0,80.0,113.0,24.0,90.0,3.0,0.0,1.0,2.0,4.0,4.0,3.0,140.0,NaN,@,31:36,8.0,13.0,0.0,0.0
19523,2019,4.0,2019-09-29,chi,min,Sun,September 29,W,16.0,6.0,24.0,33.0,197.0,1.0,0.0,1.0,7.0,6.0,5.0,72.0,97.0,33.0,72.0,2.0,0.0,3.0,3.0,1.0,1.0,5.0,241.0,NaN,NaN,35:27,5.0,16.0,2.0,3.0
19524,2019,5.0,2019-10-06,chi,rai,Sun,October 6,L,21.0,24.0,22.0,30.0,194.0,2.0,2.0,4.0,37.0,7.0,5.0,73.0,84.0,17.0,42.0,2.0,1.0,0.0,0.0,3.0,3.0,5.0,213.0,NaN,@,25:17,4.0,11.0,0.0,0.0
19525,2019,7.0,2019-10-20,chi,nor,Sun,October 20,L,25.0,36.0,34.0,54.0,235.0,2.0,0.0,2.0,16.0,4.0,4.0,63.0,85.0,7.0,17.0,2.0,0.0,1.0,1.0,2.0,2.0,7.0,263.0,NaN,NaN,22:34,2.0,12.0,1.0,2.0
19526,2019,8.0,2019-10-27,chi,sdg,Sun,October 27,L,16.0,17.0,23.0,35.0,226.0,0.0,1.0,4.0,27.0,7.0,5.0,65.0,71.0,38.0,162.0,4.0,1.0,3.0,5.0,1.0,1.0,3.0,123.0,NaN,NaN,38:00,5.0,14.0,1.0,1.0
19527,2019,9.0,2019-11-03,chi,phi,Sun,November 3,L,14.0,22.0,10.0,21.0,102.0,0.0,0.0,3.0,23.0,6.0,4.0,47.0,62.0,18.0,62.0,3.0,2.0,0.0,0.0,2.0,2.0,8.0,372.0,NaN,@,19:42,2.0,10.0,0.0,0.0
19528,2019,10.0,2019-11-10,chi,det,Sun,November 10,W,20.0,13.0,16.0,23.0,145.0,3.0,0.0,5.0,28.0,7.0,5.0,69.0,125.0,24.0,81.0,3.0,0.0,0.0,0.0,2.0,3.0,9.0,387.0,NaN,NaN,28:06,2.0,12.0,1.0,1.0
19529,2019,11.0,2019-11-17,chi,ram,Sun,November 17,L,7.0,17.0,25.0,47.0,193.0,1.0,1.0,3.0,6.0,4.0,3.0,53.0,61.0,24.0,74.0,3.0,0.0,0.0,2.0,1.0,1.0,6.0,258.0,NaN,@,32:44,6.0,17.0,0.0,2.0


In [8]:
# clean the game_location column & apply change. def home_game(row):
def apply_home_game(row):    
    if row == '@':
        return 0
    return 1

game_df['team_home_game'] = game_df.game_location.apply(apply_home_game)
game_df.drop("game_location", axis=1, errors='ignore', inplace=True)

In [9]:
# add team_year identifier to each row (to easily groupby team/year)

team_years = game_df['team'] + '-' +  game_df['season_year'].astype(str) 
game_df.insert(loc=2, column='team_year', value=team_years)
game_df.sample()

,season_year,week_num,team_year,full_game_date,team,opp,game_day_of_week,game_date,game_outcome,pts_off,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att,team_home_game
15800,2012,16.0,chi-2012,2012-12-23,chi,crd,Sun,December 23,W,28.0,13.0,12.0,26.0,145.0,1.0,0.0,1.0,1.0,5.0,5.0,46.0,76.0,33.0,152.0,4.0,1.0,0.0,1.0,4.0,4.0,9.0,394.0,NaN,28:05,3.0,13.0,0.0,0.0,0


Datetime formatting strings:

- ``%B`` month fullname
- ``%-d`` day of month without leading zero\*
    - \*note: the ``-`` only works on Linux/OS-X...for windows replace with '#'
- ``%Y`` full year

In [10]:
# convert game date to datetime & add new column to DF


# f = r"%B %d-%Y"
# full_game_date = game_df.game_date + '-' + game_df.year.astype(str)
# full_game_date = pd.to_datetime(full_game_date, format=f)
# game_df.insert(loc=2, column='date', value=full_game_date)
# game_df.sample(3)

In [11]:
# now drop the game_date col
game_df.drop("game_date", axis=1, inplace=True, errors='ignore')
game_df.sample(2) # to confirm

,season_year,week_num,team_year,full_game_date,team,opp,game_day_of_week,game_outcome,pts_off,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att,team_home_game
18376,2017,7.0,crd-2017,2017-10-22,crd,ram,Sun,L,0.0,33.0,15.0,32.0,171.0,0.0,2.0,3.0,17.0,5.0,4.0,46.0,37.0,12.0,25.0,2.0,0.0,0.0,1.0,0.0,0.0,6.0,263.0,NaN,20:59,3.0,12.0,1.0,2.0,0
17044,2014,7.0,mia-2014,2014-10-19,mia,chi,Sun,W,27.0,14.0,25.0,32.0,256.0,2.0,0.0,4.0,21.0,8.0,7.0,78.0,120.0,33.0,137.0,4.0,1.0,2.0,4.0,3.0,3.0,2.0,75.0,NaN,37:22,4.0,11.0,1.0,1.0,0


In [12]:
# add decade column
decades = game_df['season_year'] // 10
game_df.insert(loc=5, column='decade', value=decades)
game_df[['full_game_date', 'decade']].sample(10)

,full_game_date,decade
18860,2017-10-29,201
6896,1995-12-24,199
6353,1994-11-13,199
16248,2013-12-22,201
14083,2008-09-14,200
14012,2008-11-30,200
8315,1998-11-29,199
16691,2013-10-17,201
17205,2014-09-21,201
4629,1990-12-22,199


In [13]:
# add unique game identifier that will be the same for both teams in the same game


def apply_game_id(row):
    teams = []
    teams.append(str(row['team']))
    teams.append(str(row['opp']))
    teams.sort()
    
    game_id = teams[0] + '-' +  teams[1] + '-' + str(row['full_game_date'])[:-9]
    return game_id

In [14]:
game_ids = game_df.apply(apply_game_id, axis=1)
game_df.insert(loc=0, column='game_id', value=game_ids)

In [15]:
game_df

,game_id,season_year,week_num,team_year,full_game_date,team,decade,opp,game_day_of_week,game_outcome,pts_off,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att,team_home_game
4448,crd-was-1990-09-09,1990,1.0,crd-1990,1990-09-09,crd,199,was,Sun,L,0.0,31.0,20.0,39.0,206.0,0.0,4.0,3.0,22.0,5.0,4.0,51.0,27.0,24.0,88.0,3.0,0.0,0.0,0.0,0.0,0.0,4.0,196.0,NaN,28:54,NaN,NaN,NaN,NaN,0
4449,crd-phi-1990-09-16,1990,2.0,crd-1990,1990-09-16,crd,199,phi,Sun,W,23.0,21.0,11.0,19.0,89.0,0.0,1.0,6.0,27.0,6.0,3.0,57.0,47.0,31.0,145.0,4.0,2.0,3.0,3.0,2.0,2.0,4.0,143.0,NaN,29:53,NaN,NaN,NaN,NaN,0
4450,crd-nor-1990-09-23,1990,3.0,crd-1990,1990-09-23,crd,199,nor,Sun,L,7.0,28.0,14.0,26.0,186.0,1.0,1.0,1.0,5.0,7.0,6.0,53.0,73.0,30.0,111.0,3.0,0.0,0.0,2.0,1.0,1.0,3.0,139.0,NaN,30:45,NaN,NaN,NaN,NaN,0
4451,crd-was-1990-09-30,1990,4.0,crd-1990,1990-09-30,crd,199,was,Sun,L,10.0,38.0,11.0,23.0,80.0,1.0,1.0,2.0,11.0,4.0,3.0,47.0,52.0,21.0,112.0,5.0,0.0,1.0,1.0,1.0,1.0,6.0,263.0,NaN,22:54,NaN,NaN,NaN,NaN,1
4452,crd-dal-1990-10-14,1990,6.0,crd-1990,1990-10-14,crd,199,dal,Sun,W,20.0,3.0,20.0,28.0,191.0,0.0,0.0,0.0,0.0,6.0,6.0,71.0,90.0,41.0,223.0,5.0,2.0,2.0,2.0,2.0,2.0,3.0,125.0,NaN,35:39,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20503,sfo-was-2020-12-13,2020,14.0,was-2020,2020-12-13,was,202,sfo,Sun,W,23.0,15.0,15.0,32.0,95.0,0.0,1.0,2.0,13.0,3.0,2.0,46.0,40.0,28.0,98.0,3.0,0.0,3.0,4.0,2.0,2.0,8.0,398.0,NaN,27:49,3.0,15.0,0.0,0.0,0
20504,sea-was-2020-12-20,2020,15.0,was-2020,2020-12-20,was,202,sea,Sun,L,15.0,20.0,38.0,55.0,269.0,1.0,2.0,4.0,26.0,5.0,4.0,69.0,70.0,20.0,84.0,4.0,1.0,1.0,1.0,0.0,1.0,4.0,182.0,NaN,32:16,10.0,17.0,0.0,1.0,1
20505,car-was-2020-12-27,2020,16.0,was-2020,2020-12-27,was,202,car,Sun,L,13.0,20.0,26.0,47.0,278.0,1.0,2.0,3.0,13.0,6.0,5.0,55.0,62.0,19.0,108.0,5.0,0.0,2.0,2.0,1.0,1.0,3.0,154.0,NaN,24:24,7.0,15.0,1.0,3.0,1
20506,phi-was-2021-01-03,2020,17.0,was-2020,2021-01-03,was,202,phi,Sun,W,20.0,14.0,22.0,32.0,145.0,2.0,2.0,3.0,17.0,5.0,4.0,68.0,73.0,27.0,103.0,3.0,0.0,2.0,2.0,2.0,2.0,6.0,261.0,NaN,30:48,6.0,15.0,0.0,0.0,0


In [16]:
game_df.columns

Index(['game_id', 'season_year', 'week_num', 'team_year', 'full_game_date',
       'team', 'decade', 'opp', 'game_day_of_week', 'game_outcome', 'pts_off',
       'pts_def', 'pass_cmp', 'pass_att', 'pass_yds', 'pass_td', 'pass_int',
       'pass_sacked', 'pass_sacked_yds', 'pass_yds_per_att',
       'pass_net_yds_per_att', 'pass_cmp_perc', 'pass_rating', 'rush_att',
       'rush_yds', 'rush_yds_per_att', 'rush_td', 'fgm', 'fga', 'xpm', 'xpa',
       'punt', 'punt_yds', 'overtime', 'time_of_poss', 'third_down_success',
       'third_down_att', 'fourth_down_success', 'fourth_down_att',
       'team_home_game'],
      dtype='object')

In [17]:
def_unnamed_cols = ['game_id', 'team', 'opp', 'pass_cmp', 'pass_att', 'pass_yds', 'pass_td', 'pass_int',
       'pass_sacked', 'pass_sacked_yds', 'pass_yds_per_att',
       'pass_net_yds_per_att', 'pass_cmp_perc', 'pass_rating', 'rush_att',
       'rush_yds', 'rush_yds_per_att', 'rush_td', 'fgm', 'fga', 'third_down_success',
       'third_down_att', 'fourth_down_success', 'fourth_down_att']

def_named_cols = [col + '_def' for col in def_unnamed_cols]

Now we're ready to pull defensive stats from the relevant row. 

In [18]:
# pull def stats from other row


game_df = game_df.merge(
            right=game_df[def_unnamed_cols],
            left_on=["game_id", "team"],
            right_on=["game_id", "opp"],
            suffixes=[None, "_def"],
        )

In [19]:
game_df.drop(columns=['team_def', 'opp_def'], axis=1, inplace=True)

In [20]:
mask = game_df["season_year"] == 1994
game_df[mask].sample()

,game_id,season_year,week_num,team_year,full_game_date,team,decade,opp,game_day_of_week,game_outcome,pts_off,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att,team_home_game,pass_cmp_def,pass_att_def,pass_yds_def,pass_td_def,pass_int_def,pass_sacked_def,pass_sacked_yds_def,pass_yds_per_att_def,pass_net_yds_per_att_def,pass_cmp_perc_def,pass_rating_def,rush_att_def,rush_yds_def,rush_yds_per_att_def,rush_td_def,fgm_def,fga_def,third_down_success_def,third_down_att_def,fourth_down_success_def,fourth_down_att_def
2326,oti-pit-1994-11-06,1994,10.0,oti-1994,1994-11-06,oti,199,pit,Sun,L,9.0,12.0,22.0,48.0,160.0,0.0,0.0,6.0,45.0,4.0,3.0,45.0,54.0,25.0,84.0,3.0,0.0,3.0,3.0,0.0,0.0,11.0,502.0,OT,34:29,3.0,18.0,1.0,1.0,1,17.0,29.0,155.0,0.0,0.0,4.0,29.0,6.0,4.0,58.0,73.0,34.0,111.0,3.0,0.0,4.0,4.0,1.0,15.0,0.0,0.0


In [21]:
game_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15368 entries, 0 to 15367
Data columns (total 61 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   game_id                   15368 non-null  object        
 1   season_year               15368 non-null  int64         
 2   week_num                  15368 non-null  float64       
 3   team_year                 15368 non-null  object        
 4   full_game_date            15368 non-null  datetime64[ns]
 5   team                      15368 non-null  object        
 6   decade                    15368 non-null  int64         
 7   opp                       15368 non-null  object        
 8   game_day_of_week          15368 non-null  object        
 9   game_outcome              15364 non-null  object        
 10  pts_off                   15364 non-null  float64       
 11  pts_def                   15364 non-null  float64       
 12  pass_cmp          

Alright, we need to fix a few things:

- remove rows missing 'opp' value
- convert the following cols to float:
    - pass_yds_per_att
    - pass_net_yds_per_att
    - rush_yds
    - rush_yds_per_att
    - convert overtime to boolean
    - convert time_of_poss to num
    

In [22]:
# remove missing 'opp' rows
game_df['opp'] = game_df.opp.astype(str)
game_df['opp'].replace({'nan': np.NaN}, inplace=True)
game_df.dropna(subset=['opp', 'game_outcome'], how='any', inplace=True)

In [23]:
game_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15364 entries, 0 to 15367
Data columns (total 61 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   game_id                   15364 non-null  object        
 1   season_year               15364 non-null  int64         
 2   week_num                  15364 non-null  float64       
 3   team_year                 15364 non-null  object        
 4   full_game_date            15364 non-null  datetime64[ns]
 5   team                      15364 non-null  object        
 6   decade                    15364 non-null  int64         
 7   opp                       15364 non-null  object        
 8   game_day_of_week          15364 non-null  object        
 9   game_outcome              15364 non-null  object        
 10  pts_off                   15364 non-null  float64       
 11  pts_def                   15364 non-null  float64       
 12  pass_cmp          

In [24]:
game_df['pass_yds_per_att'].replace({str('-0'):0.0}, inplace=True)
game_df['pass_net_yds_per_att'].replace({str('-0'):0.0}, inplace=True)
game_df['pass_yds_per_att']

0        5.0
1        6.0
2        7.0
3        4.0
4        6.0
        ... 
15363    3.0
15364    5.0
15365    6.0
15366    5.0
15367    7.0
Name: pass_yds_per_att, Length: 15364, dtype: float64

In [25]:
game_df[['pass_yds', 'pass_yds_per_att', 'pass_net_yds_per_att', 'rush_yds', 'rush_yds_per_att']] = \
game_df[['pass_yds', 'pass_yds_per_att', 'pass_net_yds_per_att', 'rush_yds', 'rush_yds_per_att']].astype(float)

In [26]:
game_df.dropna(subset=['third_down_success', 'third_down_att', 'fourth_down_success', 'fourth_down_att'], how='any', inplace=True)

game_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14785 entries, 470 to 15367
Data columns (total 61 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   game_id                   14785 non-null  object        
 1   season_year               14785 non-null  int64         
 2   week_num                  14785 non-null  float64       
 3   team_year                 14785 non-null  object        
 4   full_game_date            14785 non-null  datetime64[ns]
 5   team                      14785 non-null  object        
 6   decade                    14785 non-null  int64         
 7   opp                       14785 non-null  object        
 8   game_day_of_week          14785 non-null  object        
 9   game_outcome              14785 non-null  object        
 10  pts_off                   14785 non-null  float64       
 11  pts_def                   14785 non-null  float64       
 12  pass_cmp        

In [27]:
# now convert overtime col to binary
game_df['overtime'].fillna(0)

470      0
471      0
472      0
473      0
474      0
        ..
15363    0
15364    0
15365    0
15366    0
15367    0
Name: overtime, Length: 14785, dtype: object

In [28]:
game_df.overtime.value_counts()

OT    889
Name: overtime, dtype: int64

In [29]:
# convert game_outcome col to binary to avoid multicolinearity

game_df[['result_tie', 'result_win']] = pd.get_dummies(game_df.game_outcome, drop_first=True)
game_df.sample(10)

,game_id,season_year,week_num,team_year,full_game_date,team,decade,opp,game_day_of_week,game_outcome,pts_off,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att,team_home_game,pass_cmp_def,pass_att_def,pass_yds_def,pass_td_def,pass_int_def,pass_sacked_def,pass_sacked_yds_def,pass_yds_per_att_def,pass_net_yds_per_att_def,pass_cmp_perc_def,pass_rating_def,rush_att_def,rush_yds_def,rush_yds_per_att_def,rush_td_def,fgm_def,fga_def,third_down_success_def,third_down_att_def,fourth_down_success_def,fourth_down_att_def,result_tie,result_win
2204,oti-phi-1994-10-24,1994,8.0,phi-1994,1994-10-24,phi,199,oti,Mon,W,21.0,6.0,13.0,24.0,287.0,2.0,1.0,4.0,23.0,12.0,10.0,54.0,107.0,30.0,85.0,2.0,1.0,0.0,0.0,3.0,3.0,5.0,206.0,NaN,28:54,8.0,16.0,0.0,1.0,1,16.0,33.0,214.0,0.0,1.0,3.0,17.0,7.0,5.0,48.0,56.0,29.0,68.0,2.0,0.0,2.0,2.0,6.0,14.0,0.0,1.0,0,1
7171,sfo-was-2004-12-18,2004,15.0,was-2004,2004-12-18,was,200,sfo,Sat,W,26.0,16.0,18.0,27.0,209.0,1.0,0.0,1.0,5.0,7.0,7.0,66.0,102.0,40.0,128.0,3.0,0.0,4.0,4.0,2.0,2.0,5.0,184.0,NaN,36:59,7.0,16.0,0.0,0.0,0,20.0,38.0,199.0,2.0,4.0,1.0,7.0,5.0,5.0,52.0,45.0,18.0,55.0,3.0,0.0,0.0,0.0,3.0,13.0,1.0,2.0,0,1
7280,cin-rav-2005-11-06,2005,9.0,cin-2005,2005-11-06,cin,200,rav,Sun,W,21.0,9.0,19.0,26.0,234.0,2.0,0.0,2.0,14.0,9.0,8.0,73.0,126.0,35.0,95.0,2.0,1.0,0.0,1.0,3.0,3.0,2.0,93.0,NaN,33:26,7.0,11.0,0.0,0.0,0,19.0,31.0,122.0,0.0,0.0,3.0,31.0,4.0,3.0,61.0,69.0,25.0,118.0,4.0,0.0,3.0,3.0,5.0,14.0,1.0,2.0,0,1
11776,chi-sfo-2014-09-14,2014,2.0,chi-2014,2014-09-14,chi,201,sfo,Sun,W,28.0,20.0,23.0,34.0,170.0,4.0,0.0,1.0,6.0,5.0,4.0,67.0,118.0,17.0,46.0,2.0,0.0,0.0,0.0,4.0,4.0,6.0,238.0,NaN,26:09,3.0,9.0,0.0,0.0,0,21.0,34.0,232.0,1.0,3.0,4.0,16.0,7.0,6.0,61.0,55.0,27.0,127.0,4.0,1.0,2.0,2.0,7.0,13.0,1.0,2.0,0,1
14064,mia-nwe-2018-09-30,2018,4.0,nwe-2018,2018-09-30,nwe,201,mia,Sun,W,38.0,7.0,23.0,35.0,274.0,3.0,2.0,0.0,0.0,7.0,7.0,65.0,94.0,40.0,175.0,4.0,2.0,1.0,1.0,5.0,5.0,2.0,80.0,NaN,36:22,10.0,15.0,0.0,0.0,1,15.0,25.0,116.0,1.0,1.0,2.0,19.0,5.0,4.0,60.0,68.0,18.0,56.0,3.0,0.0,0.0,0.0,3.0,11.0,1.0,1.0,0,1
5596,nyj-sfo-2001-10-01,2001,3.0,sfo-2001,2001-10-01,sfo,200,nyj,Mon,W,19.0,17.0,16.0,20.0,126.0,1.0,0.0,2.0,15.0,7.0,5.0,80.0,109.0,43.0,233.0,5.0,0.0,4.0,4.0,1.0,1.0,2.0,78.0,NaN,36:08,6.0,11.0,0.0,0.0,0,12.0,25.0,186.0,0.0,0.0,2.0,16.0,8.0,6.0,48.0,73.0,23.0,82.0,3.0,2.0,1.0,2.0,5.0,11.0,0.0,0.0,0,1
14667,chi-rai-2019-10-06,2019,5.0,rai-2019,2019-10-06,rai,201,chi,Sun,W,24.0,21.0,25.0,32.0,229.0,0.0,0.0,0.0,0.0,7.0,7.0,78.0,96.0,39.0,169.0,4.0,3.0,1.0,1.0,3.0,3.0,4.0,170.0,NaN,34:43,6.0,12.0,1.0,1.0,1,22.0,30.0,194.0,2.0,2.0,4.0,37.0,7.0,5.0,73.0,84.0,17.0,42.0,2.0,1.0,0.0,0.0,4.0,11.0,0.0,0.0,0,1
6962,gnb-min-2005-01-09,2004,18.0,min-2004,2005-01-09,min,200,gnb,Sun,W,31.0,17.0,19.0,29.0,261.0,4.0,0.0,4.0,23.0,9.0,7.0,65.0,133.0,26.0,123.0,4.0,0.0,1.0,2.0,4.0,4.0,4.0,52.0,NaN,29:48,5.0,13.0,1.0,2.0,0,22.0,33.0,201.0,1.0,4.0,2.0,15.0,6.0,5.0,66.0,53.0,29.0,105.0,3.0,1.0,1.0,2.0,3.0,10.0,0.0,0.0,0,1
3914,crd-det-1998-11-01,1998,9.0,det-1998,1998-11-01,det,199,crd,Sun,L,15.0,17.0,20.0,32.0,167.0,0.0,3.0,5.0,23.0,5.0,4.0,62.0,36.0,29.0,113.0,3.0,1.0,3.0,3.0,0.0,0.0,3.0,129.0,NaN,32:50,6.0,14.0,0.0,1.0,1,15.0,25.0,172.0,2.0,1.0,4.0,26.0,7.0,5.0,60.0,90.0,25.0,37.0,1.0,0.0,1.0,2.0,2.0,13.0,0.0,0.0,0,0
7429,dal-kan-2005-12-11,2005,14.0,kan-2005,2005-12-11,kan,200,dal,Sun,L,28.0,31.0,20.0,32.0,332.0,1.0,0.0,2.0,8.0,10.0,9.0,62.0,107.0,28.0,161.0,5.0,3.0,0.0,1.0,4.0,4.0,5.0,193.0,NaN,28:12,7.0,12.0,0.0,0.0,0,22.0,34.0,316.0,3.0,0.0,4.0,16.0,9.0,8.0,64.0,124.0,28.0,129.0,4.0,1.0,1.0,1.0,9.0,16.0,0.0,0.0,0,0


Okay, almost there. Our ties are coming through, but we get NaNs for teams that haven't tied, so we'll need to replace those with zeros.

In [30]:
# add 'margin' col
margins = game_df['pts_off'] - game_df['pts_def']
game_df.insert(loc=11, column='margin', value=margins)
game_df[mask].head()

<ipython-input-30-7c1aea470a18>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  game_df[mask].head()


,game_id,season_year,week_num,team_year,full_game_date,team,decade,opp,game_day_of_week,game_outcome,pts_off,margin,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att,team_home_game,pass_cmp_def,pass_att_def,pass_yds_def,pass_td_def,pass_int_def,pass_sacked_def,pass_sacked_yds_def,pass_yds_per_att_def,pass_net_yds_per_att_def,pass_cmp_perc_def,pass_rating_def,rush_att_def,rush_yds_def,rush_yds_per_att_def,rush_td_def,fgm_def,fga_def,third_down_success_def,third_down_att_def,fourth_down_success_def,fourth_down_att_def,result_tie,result_win
1880,crd-ram-1994-09-04,1994,1.0,crd-1994,1994-09-04,crd,199,ram,Sun,L,12.0,-2.0,14.0,18.0,40.0,128.0,1.0,2.0,4.0,30.0,4.0,2.0,45.0,40.0,32.0,106.0,3.0,0.0,2.0,2.0,0.0,0.0,6.0,221.0,NaN,38:22,3.0,13.0,2.0,2.0,0,6.0,16.0,102.0,0.0,1.0,5.0,29.0,8.0,4.0,37.0,33.0,22.0,50.0,2.0,1.0,0.0,1.0,2.0,11.0,0.0,0.0,0,0
1881,crd-nyg-1994-09-11,1994,2.0,crd-1994,1994-09-11,crd,199,nyg,Sun,L,17.0,-3.0,20.0,14.0,27.0,135.0,1.0,2.0,3.0,23.0,5.0,4.0,51.0,47.0,16.0,39.0,2.0,1.0,1.0,1.0,2.0,2.0,5.0,228.0,NaN,24:45,4.0,10.0,0.0,0.0,1,11.0,17.0,88.0,2.0,1.0,4.0,31.0,7.0,4.0,64.0,92.0,41.0,118.0,2.0,1.0,0.0,0.0,6.0,14.0,0.0,0.0,0,0
1882,cle-crd-1994-09-18,1994,3.0,crd-1994,1994-09-18,crd,199,cle,Sun,L,0.0,-32.0,32.0,26.0,58.0,255.0,0.0,3.0,2.0,14.0,4.0,4.0,44.0,36.0,17.0,63.0,3.0,0.0,0.0,2.0,0.0,0.0,7.0,282.0,NaN,28:20,6.0,17.0,0.0,1.0,0,17.0,27.0,243.0,2.0,1.0,1.0,5.0,9.0,8.0,63.0,101.0,27.0,79.0,2.0,1.0,1.0,1.0,2.0,9.0,0.0,1.0,0,0
1883,crd-min-1994-10-02,1994,5.0,crd-1994,1994-10-02,crd,199,min,Sun,W,17.0,10.0,7.0,20.0,36.0,200.0,1.0,1.0,2.0,21.0,6.0,5.0,55.0,69.0,30.0,109.0,3.0,1.0,1.0,2.0,2.0,2.0,5.0,200.0,NaN,32:39,7.0,14.0,0.0,0.0,1,29.0,47.0,340.0,1.0,2.0,1.0,15.0,7.0,7.0,61.0,73.0,13.0,18.0,1.0,0.0,0.0,2.0,3.0,9.0,0.0,1.0,0,1
1884,crd-dal-1994-10-09,1994,6.0,crd-1994,1994-10-09,crd,199,dal,Sun,L,3.0,-35.0,38.0,16.0,35.0,168.0,0.0,5.0,1.0,1.0,4.0,4.0,45.0,20.0,18.0,53.0,2.0,0.0,1.0,1.0,0.0,0.0,5.0,204.0,NaN,24:45,2.0,12.0,0.0,1.0,0,19.0,28.0,273.0,2.0,0.0,0.0,0.0,9.0,9.0,67.0,123.0,36.0,78.0,2.0,2.0,1.0,2.0,4.0,12.0,0.0,0.0,0,0


In [31]:
# drop rows for bye weeks
game_df.dropna(axis=0, how='any', subset=['game_outcome'], inplace=True)


In [32]:
mask = game_df['team_year'] == 'kan-2019'
game_df[mask]

,game_id,season_year,week_num,team_year,full_game_date,team,decade,opp,game_day_of_week,game_outcome,pts_off,margin,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att,team_home_game,pass_cmp_def,pass_att_def,pass_yds_def,pass_td_def,pass_int_def,pass_sacked_def,pass_sacked_yds_def,pass_yds_per_att_def,pass_net_yds_per_att_def,pass_cmp_perc_def,pass_rating_def,rush_att_def,rush_yds_def,rush_yds_per_att_def,rush_td_def,fgm_def,fga_def,third_down_success_def,third_down_att_def,fourth_down_success_def,fourth_down_att_def,result_tie,result_win
14544,jax-kan-2019-09-08,2019,1.0,kan-2019,2019-09-08,kan,201,jax,Sun,W,40.0,14.0,26.0,25.0,34.0,378.0,3.0,0.0,0.0,0.0,11.0,11.0,73.0,139.0,26.0,113.0,4.0,1.0,4.0,4.0,4.0,4.0,1.0,51.0,NaN,31:15,5.0,10.0,0.0,0.0,0,27.0,33.0,347.0,3.0,1.0,1.0,3.0,10.0,10.0,81.0,128.0,16.0,81.0,5.0,0.0,2.0,2.0,5.0,10.0,0.0,0.0,0,1
14545,kan-rai-2019-09-15,2019,2.0,kan-2019,2019-09-15,kan,201,rai,Sun,W,28.0,18.0,10.0,30.0,44.0,436.0,4.0,0.0,2.0,7.0,10.0,9.0,68.0,130.0,22.0,31.0,1.0,0.0,0.0,0.0,4.0,4.0,5.0,196.0,NaN,32:35,8.0,14.0,0.0,0.0,0,23.0,38.0,178.0,1.0,2.0,3.0,20.0,5.0,4.0,60.0,58.0,19.0,129.0,6.0,0.0,1.0,1.0,6.0,14.0,1.0,1.0,0,1
14546,kan-rav-2019-09-22,2019,3.0,kan-2019,2019-09-22,kan,201,rav,Sun,W,33.0,5.0,28.0,27.0,37.0,363.0,3.0,0.0,1.0,11.0,10.0,9.0,73.0,130.0,25.0,140.0,5.0,1.0,2.0,2.0,3.0,4.0,2.0,88.0,NaN,27:16,5.0,9.0,0.0,1.0,1,22.0,43.0,249.0,0.0,0.0,3.0,18.0,6.0,5.0,51.0,68.0,32.0,203.0,6.0,4.0,1.0,1.0,5.0,13.0,3.0,4.0,0,1
14547,det-kan-2019-09-29,2019,4.0,kan-2019,2019-09-29,kan,201,det,Sun,W,34.0,4.0,30.0,24.0,42.0,315.0,0.0,0.0,0.0,0.0,7.0,7.0,57.0,81.0,25.0,123.0,4.0,3.0,2.0,3.0,4.0,4.0,2.0,91.0,NaN,26:05,4.0,9.0,1.0,1.0,0,21.0,34.0,261.0,3.0,0.0,4.0,30.0,8.0,6.0,61.0,115.0,35.0,186.0,5.0,0.0,3.0,3.0,5.0,13.0,0.0,0.0,0,1
14548,clt-kan-2019-10-06,2019,5.0,kan-2019,2019-10-06,kan,201,clt,Sun,L,13.0,-6.0,19.0,22.0,39.0,288.0,1.0,0.0,4.0,33.0,8.0,6.0,56.0,88.0,14.0,36.0,2.0,0.0,2.0,2.0,1.0,1.0,4.0,155.0,NaN,22:45,4.0,12.0,1.0,2.0,1,18.0,29.0,151.0,0.0,1.0,0.0,0.0,5.0,5.0,62.0,61.0,45.0,180.0,4.0,1.0,4.0,4.0,5.0,14.0,2.0,2.0,0,0
14549,htx-kan-2019-10-13,2019,6.0,kan-2019,2019-10-13,kan,201,htx,Sun,L,24.0,-7.0,31.0,19.0,35.0,256.0,3.0,1.0,1.0,17.0,7.0,7.0,54.0,94.0,11.0,53.0,4.0,0.0,1.0,2.0,3.0,3.0,2.0,119.0,NaN,20:12,4.0,8.0,0.0,0.0,1,30.0,42.0,280.0,1.0,2.0,0.0,0.0,6.0,6.0,71.0,77.0,41.0,192.0,4.0,3.0,1.0,2.0,5.0,12.0,2.0,3.0,0,0
14550,den-kan-2019-10-17,2019,7.0,kan-2019,2019-10-17,kan,201,den,Thu,W,30.0,24.0,6.0,20.0,30.0,191.0,2.0,0.0,1.0,2.0,6.0,6.0,66.0,106.0,27.0,80.0,3.0,0.0,3.0,3.0,3.0,3.0,6.0,266.0,NaN,32:13,5.0,15.0,1.0,1.0,0,21.0,34.0,134.0,0.0,0.0,9.0,79.0,6.0,3.0,61.0,70.0,21.0,71.0,3.0,1.0,0.0,1.0,1.0,13.0,1.0,3.0,0,1
14551,gnb-kan-2019-10-27,2019,8.0,kan-2019,2019-10-27,kan,201,gnb,Sun,L,24.0,-7.0,31.0,24.0,36.0,249.0,2.0,0.0,2.0,18.0,7.0,6.0,66.0,105.0,20.0,88.0,4.0,1.0,1.0,2.0,3.0,3.0,3.0,154.0,NaN,26:47,6.0,11.0,0.0,0.0,1,23.0,33.0,256.0,3.0,0.0,5.0,49.0,9.0,6.0,69.0,122.0,26.0,118.0,4.0,1.0,1.0,1.0,8.0,13.0,0.0,0.0,0,0
14552,kan-min-2019-11-03,2019,9.0,kan-2019,2019-11-03,kan,201,min,Sun,W,26.0,3.0,23.0,25.0,35.0,230.0,1.0,0.0,5.0,45.0,7.0,5.0,71.0,98.0,18.0,147.0,8.0,1.0,4.0,4.0,2.0,2.0,5.0,251.0,NaN,31:48,4.0,13.0,1.0,1.0,1,19.0,38.0,212.0,3.0,0.0,1.0,8.0,5.0,5.0,50.0,93.0,27.0,96.0,3.0,0.0,1.0,1.0,5.0,15.0,1.0,1.0,0,1
14553,kan-oti-2019-11-10,2019,10.0,kan-2019,2019-11-10,kan,201,oti,Sun,L,32.0,-3.0,35.0,36.0,51.0,433.0,3.0,0.0,2.0,13.0,8.0,8.0,70.0,115.0,25.0,97.0,3.0,0.0,4.0,5.0,2.0,3.0,1.0,26.0,NaN,37:52,7.0,14.0,1.0,2.0,0,13.0,19.0,146.0,2.0,0.0,4.0,35.0,9.0,6.0,68.0,126.0,26.0,225.0,8.0,2.0,0.0,0.0,2.0,8.0,0.0,1.0,0,0


In [33]:
# add 'prev_week_num' col
game_df['prev_week_num'] = game_df.groupby('team_year')['week_num'].apply(lambda grp: grp.shift(1))

# add 'prev_result_win' col
game_df['prev_result_win'] = game_df.groupby('team_year')['result_win'].apply(lambda grp: grp.shift(1))

In [34]:
# add 'off_bye' (boolean) col
    
def apply_off_bye(row):
    if row['week_num'] > 1: # playoff games have week_num = 0
        off_bye = row['week_num'] - row['prev_week_num'] == 2
    else:
        off_bye = False
    if off_bye == True:
        return 1
    return 0


game_df['off_bye'] = game_df[['week_num', 'prev_week_num']].apply(apply_off_bye, axis=1)



In [35]:
mask = game_df.team_year == 'chi-2019'
game_df[mask]

,game_id,season_year,week_num,team_year,full_game_date,team,decade,opp,game_day_of_week,game_outcome,pts_off,margin,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att,team_home_game,pass_cmp_def,pass_att_def,pass_yds_def,pass_td_def,pass_int_def,pass_sacked_def,pass_sacked_yds_def,pass_yds_per_att_def,pass_net_yds_per_att_def,pass_cmp_perc_def,pass_rating_def,rush_att_def,rush_yds_def,rush_yds_per_att_def,rush_td_def,fgm_def,fga_def,third_down_success_def,third_down_att_def,fourth_down_success_def,fourth_down_att_def,result_tie,result_win,prev_week_num,prev_result_win,off_bye
14380,chi-gnb-2019-09-05,2019,1.0,chi-2019,2019-09-05,chi,201,gnb,Thu,L,3.0,-7.0,10.0,26.0,45.0,208.0,0.0,1.0,5.0,20.0,5.0,4.0,57.0,60.0,15.0,46.0,3.0,0.0,1.0,1.0,0.0,0.0,8.0,341.0,NaN,28:57,3.0,15.0,0.0,2.0,1,18.0,30.0,166.0,1.0,0.0,5.0,37.0,6.0,4.0,60.0,86.0,22.0,47.0,2.0,0.0,1.0,1.0,2.0,12.0,0.0,0.0,0,0,NaN,NaN,0
14381,chi-den-2019-09-15,2019,2.0,chi-2019,2019-09-15,chi,201,den,Sun,W,16.0,2.0,14.0,16.0,27.0,120.0,0.0,0.0,0.0,0.0,4.0,4.0,59.0,70.0,29.0,153.0,5.0,1.0,3.0,3.0,1.0,1.0,5.0,287.0,NaN,27:01,3.0,11.0,1.0,1.0,0,35.0,50.0,282.0,1.0,1.0,2.0,10.0,5.0,5.0,70.0,82.0,24.0,90.0,3.0,0.0,2.0,2.0,3.0,14.0,3.0,3.0,0,1,1.0,0.0,0
14382,chi-was-2019-09-23,2019,3.0,chi-2019,2019-09-23,chi,201,was,Mon,W,31.0,16.0,15.0,25.0,31.0,208.0,3.0,1.0,3.0,23.0,7.0,6.0,80.0,113.0,24.0,90.0,3.0,0.0,1.0,2.0,4.0,4.0,3.0,140.0,NaN,31:36,8.0,13.0,0.0,0.0,0,30.0,43.0,287.0,2.0,3.0,4.0,45.0,7.0,6.0,69.0,74.0,21.0,69.0,3.0,0.0,1.0,2.0,3.0,9.0,2.0,3.0,0,1,2.0,1.0,0
14383,chi-min-2019-09-29,2019,4.0,chi-2019,2019-09-29,chi,201,min,Sun,W,16.0,10.0,6.0,24.0,33.0,197.0,1.0,0.0,1.0,7.0,6.0,5.0,72.0,97.0,33.0,72.0,2.0,0.0,3.0,3.0,1.0,1.0,5.0,241.0,NaN,35:27,5.0,16.0,2.0,3.0,1,27.0,36.0,182.0,0.0,0.0,6.0,51.0,6.0,4.0,75.0,85.0,16.0,40.0,2.0,1.0,0.0,0.0,5.0,13.0,1.0,2.0,0,1,3.0,1.0,0
14384,chi-rai-2019-10-06,2019,5.0,chi-2019,2019-10-06,chi,201,rai,Sun,L,21.0,-3.0,24.0,22.0,30.0,194.0,2.0,2.0,4.0,37.0,7.0,5.0,73.0,84.0,17.0,42.0,2.0,1.0,0.0,0.0,3.0,3.0,5.0,213.0,NaN,25:17,4.0,11.0,0.0,0.0,0,25.0,32.0,229.0,0.0,0.0,0.0,0.0,7.0,7.0,78.0,96.0,39.0,169.0,4.0,3.0,1.0,1.0,6.0,12.0,1.0,1.0,0,0,4.0,1.0,0
14385,chi-nor-2019-10-20,2019,7.0,chi-2019,2019-10-20,chi,201,nor,Sun,L,25.0,-11.0,36.0,34.0,54.0,235.0,2.0,0.0,2.0,16.0,4.0,4.0,63.0,85.0,7.0,17.0,2.0,0.0,1.0,1.0,2.0,2.0,7.0,263.0,NaN,22:34,2.0,12.0,1.0,2.0,1,23.0,38.0,273.0,2.0,0.0,1.0,8.0,7.0,7.0,60.0,100.0,35.0,151.0,4.0,2.0,2.0,4.0,7.0,15.0,0.0,0.0,0,0,5.0,0.0,1
14386,chi-sdg-2019-10-27,2019,8.0,chi-2019,2019-10-27,chi,201,sdg,Sun,L,16.0,-1.0,17.0,23.0,35.0,226.0,0.0,1.0,4.0,27.0,7.0,5.0,65.0,71.0,38.0,162.0,4.0,1.0,3.0,5.0,1.0,1.0,3.0,123.0,NaN,38:00,5.0,14.0,1.0,1.0,1,19.0,29.0,195.0,1.0,1.0,1.0,6.0,6.0,6.0,65.0,81.0,12.0,36.0,3.0,1.0,1.0,2.0,2.0,10.0,0.0,0.0,0,0,7.0,0.0,0
14387,chi-phi-2019-11-03,2019,9.0,chi-2019,2019-11-03,chi,201,phi,Sun,L,14.0,-8.0,22.0,10.0,21.0,102.0,0.0,0.0,3.0,23.0,6.0,4.0,47.0,62.0,18.0,62.0,3.0,2.0,0.0,0.0,2.0,2.0,8.0,372.0,NaN,19:42,2.0,10.0,0.0,0.0,0,26.0,39.0,227.0,1.0,0.0,4.0,12.0,6.0,5.0,66.0,90.0,35.0,146.0,4.0,1.0,3.0,3.0,8.0,18.0,1.0,1.0,0,0,8.0,0.0,0
14388,chi-det-2019-11-10,2019,10.0,chi-2019,2019-11-10,chi,201,det,Sun,W,20.0,7.0,13.0,16.0,23.0,145.0,3.0,0.0,5.0,28.0,7.0,5.0,69.0,125.0,24.0,81.0,3.0,0.0,0.0,0.0,2.0,3.0,9.0,387.0,NaN,28:06,2.0,12.0,1.0,1.0,1,27.0,46.0,259.0,1.0,1.0,2.0,10.0,5.0,5.0,58.0,72.0,27.0,98.0,3.0,0.0,2.0,2.0,7.0,18.0,0.0,2.0,0,1,9.0,0.0,0
14389,chi-ram-2019-11-17,2019,11.0,chi-2019,2019-11-17,chi,201,ram,Sun,L,7.0,-10.0,17.0,25.0,47.0,193.0,1.0,1.0,3.0,6.0,4.0,3.0,53.0,61.0,24.0,74.0,3.0,0.0,0.0,2.0,1.0,1.0,6.0,258.0,NaN,32:44,6.0,17.0,0.0,2.0,0,11.0,18.0,173.0,0.0,1.0,0.0,0.0,9.0,9.0,61.0,69.0,34.0,110.0,3.0,2.0,1.0,1.0,3

In [36]:
game_df.columns

Index(['game_id', 'season_year', 'week_num', 'team_year', 'full_game_date',
       'team', 'decade', 'opp', 'game_day_of_week', 'game_outcome', 'pts_off',
       'margin', 'pts_def', 'pass_cmp', 'pass_att', 'pass_yds', 'pass_td',
       'pass_int', 'pass_sacked', 'pass_sacked_yds', 'pass_yds_per_att',
       'pass_net_yds_per_att', 'pass_cmp_perc', 'pass_rating', 'rush_att',
       'rush_yds', 'rush_yds_per_att', 'rush_td', 'fgm', 'fga', 'xpm', 'xpa',
       'punt', 'punt_yds', 'overtime', 'time_of_poss', 'third_down_success',
       'third_down_att', 'fourth_down_success', 'fourth_down_att',
       'team_home_game', 'pass_cmp_def', 'pass_att_def', 'pass_yds_def',
       'pass_td_def', 'pass_int_def', 'pass_sacked_def', 'pass_sacked_yds_def',
       'pass_yds_per_att_def', 'pass_net_yds_per_att_def', 'pass_cmp_perc_def',
       'pass_rating_def', 'rush_att_def', 'rush_yds_def',
       'rush_yds_per_att_def', 'rush_td_def', 'fgm_def', 'fga_def',
       'third_down_success_def', 'third_d

In [37]:
game_df.sort_values(['season_year', 'team', 'week_num'], inplace=True)

mask = (game_df['week_num'] == 1) & (game_df['team'] == 'chi')
game_df[mask]

,game_id,season_year,week_num,team_year,full_game_date,team,decade,opp,game_day_of_week,game_outcome,pts_off,margin,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att,team_home_game,pass_cmp_def,pass_att_def,pass_yds_def,pass_td_def,pass_int_def,pass_sacked_def,pass_sacked_yds_def,pass_yds_per_att_def,pass_net_yds_per_att_def,pass_cmp_perc_def,pass_rating_def,rush_att_def,rush_yds_def,rush_yds_per_att_def,rush_td_def,fgm_def,fga_def,third_down_success_def,third_down_att_def,fourth_down_success_def,fourth_down_att_def,result_tie,result_win,prev_week_num,prev_result_win,off_bye
523,chi-min-1991-09-01,1991,1.0,chi-1991,1991-09-01,chi,199,min,Sun,W,10.0,4.0,6.0,17.0,24.0,186.0,1.0,0.0,0.0,0.0,7.0,7.0,70.0,107.0,33.0,90.0,2.0,0.0,1.0,2.0,1.0,1.0,5.0,200.0,NaN,34:58,5.0,13.0,0.0,0.0,1,21.0,37.0,245.0,0.0,3.0,1.0,3.0,6.0,6.0,56.0,43.0,20.0,86.0,4.0,0.0,2.0,2.0,6.0,15.0,1.0,2.0,0,1,NaN,NaN,0
992,chi-det-1992-09-06,1992,1.0,chi-1992,1992-09-06,chi,199,det,Sun,W,27.0,3.0,24.0,19.0,30.0,227.0,2.0,0.0,0.0,0.0,7.0,7.0,63.0,108.0,29.0,140.0,4.0,1.0,2.0,3.0,3.0,3.0,4.0,173.0,NaN,30:51,4.0,12.0,1.0,1.0,1,18.0,26.0,232.0,2.0,1.0,6.0,41.0,10.0,7.0,69.0,106.0,23.0,121.0,5.0,1.0,1.0,1.0,3.0,10.0,0.0,0.0,0,1,NaN,NaN,0
1461,chi-nyg-1993-09-05,1993,1.0,chi-1993,1993-09-05,chi,199,nyg,Sun,L,20.0,-6.0,26.0,16.0,28.0,166.0,1.0,1.0,4.0,12.0,6.0,5.0,57.0,71.0,25.0,103.0,4.0,0.0,2.0,2.0,2.0,2.0,5.0,199.0,NaN,26:08,5.0,12.0,0.0,0.0,1,24.0,34.0,256.0,2.0,0.0,4.0,21.0,8.0,6.0,70.0,111.0,28.0,105.0,3.0,0.0,4.0,4.0,6.0,15.0,0.0,0.0,0,0,NaN,NaN,0
1928,chi-tam-1994-09-04,1994,1.0,chi-1994,1994-09-04,chi,199,tam,Sun,W,21.0,12.0,9.0,18.0,25.0,204.0,2.0,0.0,2.0,8.0,8.0,7.0,72.0,122.0,25.0,66.0,2.0,1.0,0.0,0.0,3.0,3.0,6.0,245.0,NaN,26:33,6.0,12.0,0.0,0.0,1,18.0,32.0,181.0,0.0,0.0,3.0,16.0,6.0,5.0,56.0,72.0,30.0,123.0,4.0,0.0,3.0,4.0,6.0,16.0,0.0,0.0,0,1,NaN,NaN,0
2412,chi-min-1995-09-03,1995,1.0,chi-1995,1995-09-03,chi,199,min,Sun,W,31.0,17.0,14.0,19.0,28.0,254.0,3.0,1.0,1.0,8.0,9.0,8.0,67.0,117.0,28.0,143.0,5.0,1.0,1.0,1.0,4.0,4.0,5.0,190.0,NaN,27:33,4.0,11.0,0.0,0.0,1,26.0,37.0,230.0,1.0,1.0,3.0,17.0,6.0,5.0,70.0,84.0,25.0,118.0,4.0,1.0,0.0,0.0,5.0,13.0,1.0,2.0,0,1,NaN,NaN,0
2897,chi-dal-1996-09-02,1996,1.0,chi-1996,1996-09-02,chi,199,dal,Mon,W,22.0,16.0,6.0,14.0,29.0,213.0,1.0,1.0,0.0,0.0,7.0,7.0,48.0,70.0,32.0,91.0,2.0,0.0,3.0,4.0,1.0,1.0,3.0,159.0,NaN,29:24,2.0,14.0,3.0,3.0,1,21.0,37.0,173.0,0.0,1.0,2.0,19.0,5.0,4.0,56.0,57.0,25.0,83.0,3.0,0.0,2.0,3.0,4.0,13.0,1.0,1.0,0,1,NaN,NaN,0
3366,chi-gnb-1997-09-01,1997,1.0,chi-1997,1997-09-01,chi,199,gnb,Mon,L,24.0,-14.0,38.0,17.0,41.0,172.0,1.0,2.0,3.0,20.0,4.0,3.0,41.0,41.0,30.0,164.0,5.0,2.0,1.0,1.0,1.0,1.0,6.0,288.0,NaN,32:47,5.0,16.0,1.0,2.0,0,15.0,22.0,208.0,2.0,1.0,2.0,18.0,10.0,8.0,68.0,109.0,31.0,107.0,3.0,1.0,3.0,3.0,2.0,9.0,0.0,0.0,0,0,NaN,NaN,0
3840,chi-jax-1998-09-06,1998,1.0,chi-1998,1998-09-06,chi,199,jax,Sun,L,23.0,-1.0,24.0,16.0,27.0,189.0,0.0,0.0,0.0,0.0,7.0,7.0,59.0,80.0,34.0,132.0,3.0,1.0,3.0,4.0,2.0,2.0,4.0,184.0,NaN,30:10,5.0,13.0,1.0,1.0,1,22.0,35.0,207.0,2.0,2.0,1.0,0.0,5.0,5.0,62.0,74.0,33.0,160.0,4.0,1.0,1.0,1.0,7.0,14.0,0.0,0.0,0,0,NaN,NaN,0
4306,chi-kan-1999-09-12,1999,1.0,chi-1999,1999-09-12,chi,199,kan,Sun,W,20.0,3.0,17.0,31.0,47.0,307.0,2.0,0.0,1.0,15.0,6.0,6.0,66.0,98.0,28.0,81.0,2.0,0.0,2.0,2.0,2.0,2.0,5.0,185.0,NaN,34:43,10.0,18.0,0.0,0.0,1,20.0,42.0,278.0,1.0,1.0,1.0,5.0,6.0,6.0,47.0,67.0,17.0,71.0,4.0,0.0,1.0,2.0,4.0,12.0,0.0,2.0,0,1,NaN,NaN,0
4790,chi-min-2000-09-03,2000,1.0,chi-2000,2000-09-03,chi,200,min,Sun,L,27.0,-3.0,30.0,27.0,41.0,272.0,2.0,0.0,2.0,18.0,7.0,6.0,65.0,100.0,25.0,153.0,6.0,1.0,2.0,2.0,3.0,3.0,4.0,124.0,NaN,32:34,9.0,15.0,0.0,0.0,0,13.0,23.0,188.0,0.0,1.0,1.0,2.0,8.0,7.0,56.0,65.0,30.0,186.0,6.0,3.0,3.0,3.0,6.0,

In [38]:
game_df.columns

Index(['game_id', 'season_year', 'week_num', 'team_year', 'full_game_date',
       'team', 'decade', 'opp', 'game_day_of_week', 'game_outcome', 'pts_off',
       'margin', 'pts_def', 'pass_cmp', 'pass_att', 'pass_yds', 'pass_td',
       'pass_int', 'pass_sacked', 'pass_sacked_yds', 'pass_yds_per_att',
       'pass_net_yds_per_att', 'pass_cmp_perc', 'pass_rating', 'rush_att',
       'rush_yds', 'rush_yds_per_att', 'rush_td', 'fgm', 'fga', 'xpm', 'xpa',
       'punt', 'punt_yds', 'overtime', 'time_of_poss', 'third_down_success',
       'third_down_att', 'fourth_down_success', 'fourth_down_att',
       'team_home_game', 'pass_cmp_def', 'pass_att_def', 'pass_yds_def',
       'pass_td_def', 'pass_int_def', 'pass_sacked_def', 'pass_sacked_yds_def',
       'pass_yds_per_att_def', 'pass_net_yds_per_att_def', 'pass_cmp_perc_def',
       'pass_rating_def', 'rush_att_def', 'rush_yds_def',
       'rush_yds_per_att_def', 'rush_td_def', 'fgm_def', 'fga_def',
       'third_down_success_def', 'third_d

In [39]:
# create columns containing team's season totals heading into the game
roll_cols = ['pts_off', 'margin', 'pts_def', 'pass_cmp', 'pass_att', 'pass_yds', 'pass_td',
             'pass_int', 'pass_sacked', 'pass_sacked_yds', 'pass_yds_per_att',
             'pass_cmp_perc', 'pass_rating', 'rush_att', 'rush_yds', 'rush_yds_per_att', 'rush_td',
             'fgm', 'fga', 'third_down_success', 'third_down_att',
             'fourth_down_success', 'fourth_down_att', 'team_home_game', 'result_tie', 'result_win']

[roll_cols.append(col) for col in def_named_cols]

roll_cols



['pts_off',
 'margin',
 'pts_def',
 'pass_cmp',
 'pass_att',
 'pass_yds',
 'pass_td',
 'pass_int',
 'pass_sacked',
 'pass_sacked_yds',
 'pass_yds_per_att',
 'pass_cmp_perc',
 'pass_rating',
 'rush_att',
 'rush_yds',
 'rush_yds_per_att',
 'rush_td',
 'fgm',
 'fga',
 'third_down_success',
 'third_down_att',
 'fourth_down_success',
 'fourth_down_att',
 'team_home_game',
 'result_tie',
 'result_win',
 'game_id_def',
 'team_def',
 'opp_def',
 'pass_cmp_def',
 'pass_att_def',
 'pass_yds_def',
 'pass_td_def',
 'pass_int_def',
 'pass_sacked_def',
 'pass_sacked_yds_def',
 'pass_yds_per_att_def',
 'pass_net_yds_per_att_def',
 'pass_cmp_perc_def',
 'pass_rating_def',
 'rush_att_def',
 'rush_yds_def',
 'rush_yds_per_att_def',
 'rush_td_def',
 'fgm_def',
 'fga_def',
 'third_down_success_def',
 'third_down_att_def',
 'fourth_down_success_def',
 'fourth_down_att_def']

In [40]:
# Add roll3_wins column

game_df['roll3_wins'] = (game_df.groupby('team_year')['result_win']
                      .transform(lambda x: round(x.shift(1).rolling(3).sum())))




In [41]:
# game_df.dropna(subset=roll_cols, how='any', inplace=True)
game_df.shape

(14785, 68)

Now let's add more columns with **Expontentially Weighted Moving Averages (EWMA)** of our stats. This is similar to taking a rolling mean of a team's previous 6 performances, but EWMA weights the most recent game in the window much higher than the week before that, and so on.

For this, we will use a 3-16 week span (greedy). My hypothesis is that the most recent performances are the best predictors, and this should help further quantify recent performance.

In [42]:
roll_cols.remove('game_id_def')
roll_cols.remove('team_def')
roll_cols.remove('opp_def')

In [43]:
# add ewma cols

ewma19_cols = ['ewma19_' + col_name for col_name in roll_cols]

game_df[ewma19_cols] = (game_df.groupby('team_year')[roll_cols]
                      .transform(lambda x: round(x.shift(1).ewm(span=19, min_periods=3).mean(), 3)))

In [44]:
np.mean([197, 120, 208])

175.0

Now, let's delete the first three rows of each team-year. We have to do this because we will be using at least some rolling columns for our model.

In [45]:
game_df.dropna(axis=0, how="any", subset=["ewma19_pass_yds"], inplace=True)

Looks good!!

**Now, we have to solve a pretty complex problem.**

Currently, each row has all the stats needed for the team in the 'team' column. However, we don't have the same information for the opponent in the same row. Let's make that happen.

To further clarify, for each row, we need to pull the opposing team's rolling stats, adding them back into the same row. This way, our model will only need one row to make a prediction.

Let's take a look at an example.

In [46]:
mask = (game_df.game_id == 'chi-rai-2019-10-06')
game_df[mask]


,game_id,season_year,week_num,team_year,full_game_date,team,decade,opp,game_day_of_week,game_outcome,pts_off,margin,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att,team_home_game,pass_cmp_def,pass_att_def,pass_yds_def,pass_td_def,pass_int_def,pass_sacked_def,pass_sacked_yds_def,pass_yds_per_att_def,pass_net_yds_per_att_def,...,ewma3_third_down_att_def,ewma3_fourth_down_success_def,ewma3_fourth_down_att_def,ewma19_pts_off,ewma19_margin,ewma19_pts_def,ewma19_pass_cmp,ewma19_pass_att,ewma19_pass_yds,ewma19_pass_td,ewma19_pass_int,ewma19_pass_sacked,ewma19_pass_sacked_yds,ewma19_pass_yds_per_att,ewma19_pass_cmp_perc,ewma19_pass_rating,ewma19_rush_att,ewma19_rush_yds,ewma19_rush_yds_per_att,ewma19_rush_td,ewma19_fgm,ewma19_fga,ewma19_third_down_success,ewma19_third_down_att,ewma19_fourth_down_success,ewma19_fourth_down_att,ewma19_team_home_game,ewma19_result_tie,ewma19_result_win,ewma19_pass_cmp_def,ewma19_pass_att_def,ewma19_pass_yds_def,ewma19_pass_td_def,ewma19_pass_int_def,ewma19_pass_sacked_def,ewma19_pass_sacked_yds_def,ewma19_pass_yds_per_att_def,ewma19_pass_net_yds_per_att_def,ewma19_pass_cmp_perc_def,ewma19_pass_rating_def,ewma19_rush_att_def,ewma19_rush_yds_def,ewma19_rush_yds_per_att_def,ewma19_rush_td_def,ewma19_fgm_def,ewma19_fga_def,ewma19_third_down_success_def,ewma19_third_down_att_def,ewma19_fourth_down_success_def,ewma19_fourth_down_att_def
14384,chi-rai-2019-10-06,2019,5.0,chi-2019,2019-10-06,chi,201,rai,Sun,L,21.0,-3.0,24.0,22.0,30.0,194.0,2.0,2.0,4.0,37.0,7.0,5.0,73.0,84.0,17.0,42.0,2.0,1.0,0.0,0.0,3.0,3.0,5.0,213.0,NaN,25:17,4.0,11.0,0.0,0.0,0.0,25.0,32.0,229.0,0.0,0.0,0.0,0.0,7.0,7.0,...,12.000,1.467,2.267,17.170,6.082,11.088,22.801,33.607,184.074,1.076,0.474,2.136,12.294,5.579,67.852,86.985,25.887,90.277,3.18,0.236,2.053,2.314,4.890,13.825,0.817,1.532,0.503,0.0,0.788,27.762,39.858,229.641,0.971,1.021,4.322,36.805,6.026,4.759,69.072,81.627,20.465,60.850,2.497,0.291,0.945,1.206,3.370,11.977,1.521,2.073
14667,chi-rai-2019-10-06,2019,5.0,rai-2019,2019-10-06,rai,201,chi,Sun,W,24.0,3.0,21.0,25.0,32.0,229.0,0.0,0.0,0.0,0.0,7.0,7.0,78.0,96.0,39.0,169.0,4.0,3.0,1.0,1.0,3.0,3.0,4.0,170.0,NaN,34:43,6.0,12.0,1.0,1.0,1.0,22.0,30.0,194.0,2.0,2.0,4.0,37.0,7.0,5.0,...,13.667,1.600,1.600,20.121,-5.742,25.863,23.253,32.374,207.790,1.552,0.733,2.044,12.038,6.874,72.095,96.013,24.950,128.855,4.55,0.715,0.738,1.000,5.772,12.633,1.050,1.311,0.448,0.0,0.503,22.422,35.807,278.069,2.288,0.291,1.107,5.676,7.599,7.151,63.921,105.396,26.690,106.213,3.264,0.785,1.450,1.953,6.209,13.061,0.872,0.872


So, in the above table, we need to get all of the roll3_x and ewma_x columns from the adjacent tables, respectively. We'll prefix these with opp_.

In [47]:
[col for col in game_df.columns]

['game_id',
 'season_year',
 'week_num',
 'team_year',
 'full_game_date',
 'team',
 'decade',
 'opp',
 'game_day_of_week',
 'game_outcome',
 'pts_off',
 'margin',
 'pts_def',
 'pass_cmp',
 'pass_att',
 'pass_yds',
 'pass_td',
 'pass_int',
 'pass_sacked',
 'pass_sacked_yds',
 'pass_yds_per_att',
 'pass_net_yds_per_att',
 'pass_cmp_perc',
 'pass_rating',
 'rush_att',
 'rush_yds',
 'rush_yds_per_att',
 'rush_td',
 'fgm',
 'fga',
 'xpm',
 'xpa',
 'punt',
 'punt_yds',
 'overtime',
 'time_of_poss',
 'third_down_success',
 'third_down_att',
 'fourth_down_success',
 'fourth_down_att',
 'team_home_game',
 'pass_cmp_def',
 'pass_att_def',
 'pass_yds_def',
 'pass_td_def',
 'pass_int_def',
 'pass_sacked_def',
 'pass_sacked_yds_def',
 'pass_yds_per_att_def',
 'pass_net_yds_per_att_def',
 'pass_cmp_perc_def',
 'pass_rating_def',
 'rush_att_def',
 'rush_yds_def',
 'rush_yds_per_att_def',
 'rush_td_def',
 'fgm_def',
 'fga_def',
 'third_down_success_def',
 'third_down_att_def',
 'fourth_down_success_de

In [48]:
# gather one list of columns to get from the other game record.


prefixes = ['prev', 'roll', 'ewma']

opp_pull_cols = [col for col in game_df.columns if col[:4] in prefixes]
# add defensive columns
[opp_pull_cols.append(col) for col in game_df.columns if col[-4:] == '_def']

opp_pull_cols = list(set(opp_pull_cols))
opp_pull_cols.sort()
opp_pull_cols.insert(0, "opp")
opp_pull_cols.insert(0, "team")
opp_pull_cols.insert(0, "game_id")
opp_pull_cols

['game_id',
 'team',
 'opp',
 'ewma19_fga',
 'ewma19_fga_def',
 'ewma19_fgm',
 'ewma19_fgm_def',
 'ewma19_fourth_down_att',
 'ewma19_fourth_down_att_def',
 'ewma19_fourth_down_success',
 'ewma19_fourth_down_success_def',
 'ewma19_margin',
 'ewma19_pass_att',
 'ewma19_pass_att_def',
 'ewma19_pass_cmp',
 'ewma19_pass_cmp_def',
 'ewma19_pass_cmp_perc',
 'ewma19_pass_cmp_perc_def',
 'ewma19_pass_int',
 'ewma19_pass_int_def',
 'ewma19_pass_net_yds_per_att_def',
 'ewma19_pass_rating',
 'ewma19_pass_rating_def',
 'ewma19_pass_sacked',
 'ewma19_pass_sacked_def',
 'ewma19_pass_sacked_yds',
 'ewma19_pass_sacked_yds_def',
 'ewma19_pass_td',
 'ewma19_pass_td_def',
 'ewma19_pass_yds',
 'ewma19_pass_yds_def',
 'ewma19_pass_yds_per_att',
 'ewma19_pass_yds_per_att_def',
 'ewma19_pts_def',
 'ewma19_pts_off',
 'ewma19_result_tie',
 'ewma19_result_win',
 'ewma19_rush_att',
 'ewma19_rush_att_def',
 'ewma19_rush_td',
 'ewma19_rush_td_def',
 'ewma19_rush_yds',
 'ewma19_rush_yds_def',
 'ewma19_rush_yds_per_a

In [49]:
game_df.opp

489      rai
490      nor
491      sfo
492      crd
493      ram
        ... 
15363    sfo
15364    sea
15365    car
15366    phi
15367    tam
Name: opp, Length: 12049, dtype: object

In [50]:
# # convert all cols to float

# game_df[opp_pull_cols] = game_df[opp_pull_cols].fillna(0)
# for col in opp_pull_cols[3:]:
#     game_df[col] = game_df[col].astype(float)

**Let's solve this using `pd.merge`, using a self-join.**

In [51]:
game_df = (game_df
             .merge(right=game_df[opp_pull_cols],
                    left_on=['game_id', 'team'],
                    right_on=['game_id', 'opp'],
                    suffixes=[None, '_opp']))

Let's make sure it worked by checking a couple of random game_ids.

In [52]:
mask = (game_df.game_id == 'dal-nyg-2019-11-04')

game_df[mask]
              

,game_id,season_year,week_num,team_year,full_game_date,team,decade,opp,game_day_of_week,game_outcome,pts_off,margin,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att,team_home_game,pass_cmp_def,pass_att_def,pass_yds_def,pass_td_def,pass_int_def,pass_sacked_def,pass_sacked_yds_def,pass_yds_per_att_def,pass_net_yds_per_att_def,...,ewma3_pass_sacked_yds_opp,ewma3_pass_sacked_yds_def_opp,ewma3_pass_td_opp,ewma3_pass_td_def_opp,ewma3_pass_yds_opp,ewma3_pass_yds_def_opp,ewma3_pass_yds_per_att_opp,ewma3_pass_yds_per_att_def_opp,ewma3_pts_def_opp,ewma3_pts_off_opp,ewma3_result_tie_opp,ewma3_result_win_opp,ewma3_rush_att_opp,ewma3_rush_att_def_opp,ewma3_rush_td_opp,ewma3_rush_td_def_opp,ewma3_rush_yds_opp,ewma3_rush_yds_def_opp,ewma3_rush_yds_per_att_opp,ewma3_rush_yds_per_att_def_opp,ewma3_team_home_game_opp,ewma3_third_down_att_opp,ewma3_third_down_att_def_opp,ewma3_third_down_success_opp,ewma3_third_down_success_def_opp,fga_def_opp,fgm_def_opp,fourth_down_att_def_opp,fourth_down_success_def_opp,pass_att_def_opp,pass_cmp_def_opp,pass_cmp_perc_def_opp,pass_int_def_opp,pass_net_yds_per_att_def_opp,pass_rating_def_opp,pass_sacked_def_opp,pass_sacked_yds_def_opp,pass_td_def_opp,pass_yds_def_opp,pass_yds_per_att_def_opp,prev_result_win_opp,prev_week_num_opp,pts_def_opp,roll3_wins_opp,rush_att_def_opp,rush_td_def_opp,rush_yds_def_opp,rush_yds_per_att_def_opp,third_down_att_def_opp,third_down_success_def_opp
11191,dal-nyg-2019-11-04,2019,9.0,dal-2019,2019-11-04,dal,201,nyg,Mon,W,37.0,19.0,18.0,22.0,35.0,257.0,3.0,1.0,0.0,0.0,7.0,7.0,62.0,101.0,30.0,172.0,5.0,0.0,3.0,4.0,4.0,4.0,2.0,80.0,NaN,28:04,4.0,11.0,0.0,0.0,0.0,26.0,41.0,171.0,1.0,1.0,5.0,39.0,5.0,3.0,...,35.537,22.384,2.522,1.702,229.243,250.553,6.322,8.392,29.424,22.137,0.0,0.047,22.514,29.906,0.325,1.157,85.431,101.831,3.325,2.961,0.353,12.043,13.82,5.647,7.027,4.0,3.0,0.0,0.0,35.0,22.0,62.0,1.0,7.0,101.0,0.0,0.0,3.0,257.0,7.0,0.0,8.0,37.0,0.0,30.0,0.0,172.0,5.0,11.0,4.0
11346,dal-nyg-2019-11-04,2019,9.0,nyg-2019,2019-11-04,nyg,201,dal,Mon,L,18.0,-19.0,37.0,26.0,41.0,171.0,1.0,1.0,5.0,39.0,5.0,3.0,63.0,70.0,22.0,100.0,4.0,0.0,4.0,4.0,0.0,1.0,3.0,142.0,NaN,31:56,5.0,16.0,1.0,1.0,1.0,22.0,35.0,257.0,3.0,1.0,0.0,0.0,7.0,7.0,...,18.866,21.087,0.898,1.047,259.173,215.709,7.890,7.606,16.780,29.583,0.0,0.559,32.457,25.661,2.291,0.780,157.409,98.748,4.339,3.465,0.669,13.606,10.15,7.370,2.913,4.0,4.0,1.0,1.0,41.0,26.0,63.0,1.0,3.0,70.0,5.0,39.0,1.0,171.0,5.0,1.0,7.0,18.0,1.0,22.0,0.0,100.0,4.0,16.0,5.0


## Now let's take another look to make sure everything looks good.

If it does, we're ready to start modeling!

In [54]:
game_df.columns

Index(['game_id', 'season_year', 'week_num', 'team_year', 'full_game_date',
       'team', 'decade', 'opp', 'game_day_of_week', 'game_outcome',
       ...
       'prev_result_win_opp', 'prev_week_num_opp', 'pts_def_opp',
       'roll3_wins_opp', 'rush_att_def_opp', 'rush_td_def_opp',
       'rush_yds_def_opp', 'rush_yds_per_att_def_opp',
       'third_down_att_def_opp', 'third_down_success_def_opp'],
      dtype='object', length=283)

In [59]:
mask = game_df['team_year'] == 'chi-2019'
game_df[mask][['team', 'opp', 'season_year', 'week_num', 'team_year', 'rush_td', 'rush_td_def', 'ewma3_rush_td', 'ewma3_rush_td_def', 'ewma3_rush_td_opp', 'ewma3_rush_td_def_opp',
               'ewma19_rush_td', 'ewma19_rush_td_def', 'ewma19_rush_td_opp', 'ewma19_rush_td_def_opp']]

,team,opp,season_year,week_num,team_year,rush_td,rush_td_def,ewma3_rush_td,ewma3_rush_td_def,ewma3_rush_td_opp,ewma3_rush_td_def_opp,ewma19_rush_td,ewma19_rush_td_def,ewma19_rush_td_opp,ewma19_rush_td_def_opp
11122,chi,min,2019,4.0,chi-2019,0.0,1.0,0.286,0.000,2.429,0.286,0.332,0.000,2.336,0.332
11123,chi,rai,2019,5.0,chi-2019,1.0,3.0,0.133,0.533,0.667,0.800,0.236,0.291,0.715,0.785
11124,chi,nor,2019,7.0,chi-2019,0.0,2.0,0.581,1.806,0.079,0.587,0.422,0.952,0.282,1.082
11125,chi,sdg,2019,8.0,chi-2019,1.0,1.0,0.286,1.905,0.087,0.921,0.332,1.176,0.355,0.747
11126,chi,phi,2019,9.0,chi-2019,2.0,1.0,0.646,1.449,1.659,0.988,0.460,1.142,1.166,0.963
11127,chi,det,2019,10.0,chi-2019,0.0,0.0,1.325,1.224,0.078,1.373,0.731,1.117,0.232,1.178
11128,chi,ram,2019,11.0,chi-2019,0.0,2.0,0.661,0.611,0.544,0.162,0.611,0.935,1.102,0.642
11129,chi,nyg,2019,12.0,chi-2019,1.0,0.0,0.330,1.306,0.081,1.289,0.517,1.098,0.466,1.203
11130,chi,det,2019,13.0,chi-2019,0.0,0.0,0.665,0.653,0.510,0.421,0.588,0.938,0.402,0.844
11131,chi,dal,2019,14.0,chi-2019,1.0,2.0,0.333,0.326,0.196,0.837,0.506,0.808,0.835,0.900


Looks great. Now it's time to start modeling our data! Head over to `model_exploration.ipynb` to continue following along.